In [6]:
# !wget --load-cookies cookies.txt  https://www.kaggle.com/START-UMD/gtd/download
# !wget --load-cookies cookies.txt https://www.kaggle.com/burhanykiyakoglu/infant-mortality-fertility-income/download
# !wget --load-cookies cookies.txt https://www.kaggle.com/ibrahimmukherjee/gdp-world-bank-data/download
# !wget --load-cookies cookies.txt https://www.kaggle.com/juanumusic/countries-iso-codes/download
# !unzip download.4
# !ls
# !pip install google-cloud-bigquery==1.23.0
# !pip install  six==1.13.0




In [7]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd 
import json 



In [8]:
# 128 * 256 * 5

In [9]:

pd.set_option('display.max_columns', 500)

In [10]:
#iso country codes 
codes = pd.read_csv('wikipedia-iso-country-codes.csv').loc[:, ['English short name lower case', 'Alpha-3 code']]
codes.columns = ['Name', 'Code']


codes.head(5)

codes[codes.Name.str.contains('United')]

,Name,Code
216,"Tanzania, United Republic of",TZA
230,United Arab Emirates,ARE
231,United Kingdom,GBR
232,United States Of America,USA
233,United States Minor Outlying Islands,UMI


In [11]:
#fix country names for terror 
terror_country_map = { 'South Korea' : 'Korea, Republic of (South Korea)'  
                     , 'United States' : 'United States Of America'
                     , 'Czechoslovakia' : 'Czech Republic'
                     , 'Brunei' : 'Brunei Darussalam'
                     , 'Soviet Union' : 'Russia'
                     }


In [12]:
#terror incidents per year by country 


terror = pd.read_csv('globalterrorismdb_0718dist.csv'
#                      , nrows = 10000
                     , encoding = "ISO-8859-1"
                    , usecols = ['iyear', 'country_txt', 'success'])

terror = terror.replace({'country_txt' : terror_country_map})
terror = terror.rename(columns = {'iyear' : 'Year'})
terror = terror.merge(codes, how = 'left', left_on = 'country_txt', right_on = 'Name')

#countries excluded 
terror.loc[terror.Code.isnull(), :].country_txt.unique()

terror = terror.loc[~terror.Code.isnull(), ['Code', 'Year', 'success']]

# terror.head(5)

terror = terror.groupby(['Code', 'Year']).agg(['count', 'sum'])
terror = terror.reset_index()
terror.columns = [''.join(col) for col in terror.columns]
terror = terror.rename(columns = {'successcount' : 'incidents', 'successsum' : 'successes'})

terror.head(5)



,Code,Year,incidents,successes
0,AFG,1973,1,0
1,AFG,1979,3,3
2,AFG,1987,1,1
3,AFG,1988,11,11
4,AFG,1989,10,6


In [13]:
#population by country by year
population = pd.read_csv('PopulationPerCountry.csv' , encoding = "ISO-8859-1", skiprows = 3)

population = pd.melt(population, ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])
# 
population = population.loc[population.value >= 0.0, ['Country Code', 'variable', 'value']]
population.columns = ['Code', 'Year', 'Population']
# population['DataType'] = 'Population'
population.Year = pd.to_numeric(population.Year)
population.head(5)





,Code,Year,Population
0,ABW,1960,54211.0
1,AFG,1960,8996351.0
2,AGO,1960,5643182.0
3,ALB,1960,1608800.0
4,AND,1960,13411.0


In [14]:
#education by country by year 

#this one from kaggle is missing data 
education = pd.read_csv('public-education-expenditure-as-share-of-gdp.csv', 
                        usecols = ['Code', 'Year', 'Public Expenditure on Education (percent of GDP)']
                       )
education.columns = ['Code', 'Year', 'EducationPctGdp']
education['DataType'] = 'EducationPctGDP'


education.head(5)


#this one is better
credentials = service_account.Credentials.from_service_account_file('gckey.json')

client = bigquery.client.Client(project = 'tpu-44747', credentials = credentials)
datasetref = client.dataset('world_bank_intl_education', project = 'bigquery-public-data')

dataset = client.get_dataset(datasetref)
    
query = """
    SELECT *
    FROM `bigquery-public-data.world_bank_intl_education.international_education`

    where indicator_name = 'Government expenditure on education as % of GDP (%)'
    and year >= 1970 
    
"""
query_job = client.query(query)  # Make an API request.

education = client.query(query).to_dataframe()
education = education.loc[:, ['country_code', 'year', 'value']]
education.columns = ['Code', 'Year', 'EducationPctGdp']

education.head(5)


,Code,Year,EducationPctGdp
0,BRN,2016,4.42541
1,RWA,2016,3.61734
2,SYC,1970,4.18848
3,TGO,1970,2.11380
4,FRA,1970,3.42152


In [15]:
#mortality, fertility, income by country by year

mort = pd.read_csv('MortalityFertilityIncome.csv', 
#                    nrows = 5, 
                   encoding = "ISO-8859-1", 
                   thousands='.', 
                   decimal=',')

mort = pd.melt(mort, ['Country Name', 'Country Code', 'Region'])


mort['Data Type'] = 'InfantMortality' 
mort.loc[mort.variable.str.contains('i'), 'Data Type'] = 'IncomePerCapita'
mort.loc[mort.variable.str.contains('f'), 'Data Type'] = 'FertilityRate'
mort.variable = mort.variable.str.slice(1)
mort = mort.rename(columns = {'variable' : 'Year', 'Country Code' : 'Code'})
mort = mort.loc[:, ['Code', 'Year' ,'value', 'Data Type']]
mort.value = pd.to_numeric(mort.value)
mort.Year = pd.to_numeric(mort.Year)
mort = pd.pivot_table(mort, index = ['Code', 'Year'], columns = 'Data Type').reset_index()
mort.columns = [''.join(col).replace('value', '') for col in mort.columns]

mort.loc[mort.Code == 'USA'].head(5)


,Code,Year,FertilityRate,IncomePerCapita,InfantMortality
9012,USA,1970,2.480,4591.218634,19.9
9013,USA,1971,2.266,4891.985558,19.1
9014,USA,1972,2.010,5305.320034,18.3
9015,USA,1973,1.879,5912.942042,17.5
9016,USA,1974,1.835,6152.478813,16.7


In [16]:

#construct the main data table by cartesian of years and codes 

years =  [i for i in range(1970,2020)]
country_codes = list(codes.Code)

index = pd.MultiIndex.from_product([country_codes, years], names = ['Code', 'Year'])
data = pd.DataFrame(index = index).reset_index()

data.head(5)

#add leaf data 

data = data.merge(terror, how = 'left', on = ['Code', 'Year'])
data = data.merge(population, how = 'left', on = ['Code', 'Year'])
data = data.merge(education, how = 'left', on = ['Code', 'Year'])
data = data.merge(mort, how = 'left', on = ['Code', 'Year'])


data.head(5)



,Code,Year,incidents,successes,Population,EducationPctGdp,FertilityRate,IncomePerCapita,InfantMortality
0,AFG,1970,NaN,NaN,11126123.0,NaN,7.45,150.567338,204.8
1,AFG,1971,NaN,NaN,11417825.0,1.16036,7.45,153.663587,201.0
2,AFG,1972,NaN,NaN,11721940.0,1.11718,7.45,130.847360,197.0
3,AFG,1973,1.0,0.0,12027822.0,1.42788,7.45,138.271770,193.1
4,AFG,1974,NaN,NaN,12321541.0,NaN,7.45,167.507869,188.8


In [17]:
#data cleanup 

#years where we have terror incident data 
#assume nans mean no incidents/sucessess at all 
print('Incident Data spans %i to %i' % (min(data.loc[~data.incidents.isnull(), 'Year'].unique()), 
      max(data.loc[~data.incidents.isnull(), 'Year'].unique() )) )
data.loc[data.incidents.isnull(), ['incidents','successes']] = 0.0 


# data.loc[data.EducationPctGdp.isnull()] 
summed = data.loc[:, ['Code', 'EducationPctGdp']].groupby(['Code']).agg('sum')
noedu = summed.loc[summed.EducationPctGdp == 0].index.values
print("countries that don't report education spending as pct of gdp" , noedu)

#keep countries that report edu data 
data = data.loc[~data.Code.isin(noedu)]
data


Incident Data spans 1970 to 2017
countries that don't report education spending as pct of gdp ['AIA' 'ALA' 'ANT' 'ASM' 'ATA' 'ATF' 'BIH' 'BLM' 'BVT' 'CCK' 'COK' 'CXR'
 'CYM' 'ESH' 'FLK' 'FRO' 'GGY' 'GIB' 'GLP' 'GRL' 'GUF' 'GUM' 'HMD' 'IMN'
 'IOT' 'JEY' 'MAF' 'MNE' 'MNP' 'MSR' 'MTQ' 'MYT' 'NCL' 'NFK' 'NIU' 'NRU'
 'PCN' 'PRK' 'PYF' 'REU' 'SGS' 'SHN' 'SJM' 'SPM' 'SUR' 'TKL' 'TWN' 'UMI'
 'UZB' 'VAT' 'VIR' 'WLF']


,Code,Year,incidents,successes,Population,EducationPctGdp,FertilityRate,IncomePerCapita,InfantMortality
0,AFG,1970,0.0,0.0,11126123.0,NaN,7.450,150.567338,204.8
1,AFG,1971,0.0,0.0,11417825.0,1.160360,7.450,153.663587,201.0
2,AFG,1972,0.0,0.0,11721940.0,1.117180,7.450,130.847360,197.0
3,AFG,1973,1.0,0.0,12027822.0,1.427880,7.450,138.271770,193.1
4,AFG,1974,0.0,0.0,12321541.0,NaN,7.450,167.507869,188.8
5,AFG,1975,0.0,0.0,12590286.0,1.303320,7.450,178.635551,184.7
6,AFG,1976,0.0,0.0,12840299.0,NaN,7.450,189.113162,180.6
7,AFG,1977,0.0,0.0,13067538.0,NaN,7.449,214.744437,176.3
8,AFG,1978,0.0,0.0,13237734.0,NaN,7.449,236.915946,172.0
9,AFG,1979,3.0,3.0,13306695.0,1.733980,7.449,263.661470,167.7


In [18]:
data.to_csv('midtermdata.csv')

Datasets Included in Analysis 

Datasets were chosen to cohesively answer the four main questions listed below for a specific country and time period, with granularity in years. This section will explain how each dataset contributes to 1 or more questions. 
1. Is infant mortality improving, stable, or getting worse? 
2. Is income rising, stagnant, or falling?
3. Does one country or another seem more of less stable than ohers and why do you say this? 
4. What changes do you predict for these countries and why? 

bigquery-public-data.world_bank_intl_education.international_education
This dataset was chosen to help answer questions 3 and 4. Education spending may suggest stability and/or affect future outcomes. 

https://www.kaggle.com/juanumusic/countries-iso-codes/download
This dataset was chosen to help all questions by providing a standardized list of country codes to align all datasets. 

https://www.kaggle.com/burhanykiyakoglu/infant-mortality-fertility-income/download
This dataset directly answers 1 and 2. It can serve as a baseline for question 4.  

Population Dataset
This dataset helps adjust other datasets by population. Namely, it is used to adjust the terrorism dataset to show per capita incidents. This helps for question 3. 

Terrorism Dataset
This dataset helps answer question 3. We can assume that fewer terrorist attacks are indicative of a more stable country. It may also impact question 4 as a predictor of future outcomes. 

Steps to Prepare Datasets 
In general, all datasets were processed into a dataframe with the fields Country Code, Year, Metric 1, Metric 2, Metric N. This output will be easier to link together. 

bigquery-public-data.world_bank_intl_education.international_education
The data needed to be queried using the bigquery api. Column names needed to be changed. No additional transformations needed as data was already in proper tabular format. Data was converted to a pandas dataframe for compatability. 

https://www.kaggle.com/burhanykiyakoglu/infant-mortality-fertility-income/download
The data needed to be read into a pandas dataframe. The years attribute was stored in the columns, rather than the rows. This was fixed by using pd.melt, which turned the years columns into a single column, with 1 row per each year. The 3 metrics were set up as 3 columns. 

Population Dataset
this dataset was already in the desired format. After reading the csv into a pandas dataframe, columns were renamed. 

Terrorism Dataset
This dataset required mapping of country names to country codes. Mapping was done using the ISO country codes with specific mappings for certain countries, such as 'Czechoslovakia' to 'Czech Republic'. Because each row represents a single incident, only incidents with a non null value for sucess (0 or 1) were included. The rows were aggregated by country and year to create a dataset that counted for each country and year the number of incidents and successful incidents. 

Steps to Link Datasets 
To simplify analysis, all datasets will use the primary keys of Country Code and Year. This means that no country and year combination can appear twice for the same metric. In order to ensure no gaps in the final dataset, a cartesian product of country codes and years (1970 to 2020) was used as the base dataset. All other datasets are joined to the base dataset via a left join on the country codes and years. The final output is a tabular dataset that does not require any additional joins. The main benefit is that gaps in the data are easy to identify, for some countries will have Nan values for certain years and actual values for other years. 

Additional metrics can be added from additional datasets, so long as the dataset is processed into the format Country Code, Year, and Metric. 
